# Working with the Scholarly Migration Database

This Python-Notebook demonstrates some simple entry-points for working with the data from the Scholarly Migration Database. This notebook runs completely in your browser - so don't be afraid you can't break anything here :)

In [1]:
import pandas as pd
from js import fetch

/lib/python3.9/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


### Importing the Data



#### Importing the country numbers

In [19]:
fn_mi = "https://tomthe.github.io/jupyterlite_migration/files/data/dfmipoppresentation.csv"
# In a normal Python-environment you could just use 
# df = pd.read_csv(fn_mi)
# but because this runs completely in your webbrowser, we have to fetch 
# the data first:
res = await fetch(fn_mi)
text = await res.text()
filename = 'dfmipop.csv'
with open(filename, 'w') as f:
    f.write(text)

dfc = pd.read_csv(filename)
del dfc["Unnamed: 0"]
dfc["countrycode"] = dfc["countrycode"].str.upper()
dfc[dfc["countrycode"]=="DEU"]

,countrycode,year,padded_population_of_researchers,number_of_inmigrations,number_of_outmigrations,netmigration,inmigrationrate,outmigrationrate,netmigrationrate,gdp_per_capita,mean_n_researchers
952,DEU,1999.0,93075.0,2436.0,2689.0,-253.0,0.026172,0.028891,-0.002718,26725.915218,123566.95
953,DEU,2000.0,96976.0,2790.0,2915.0,-125.0,0.028770,0.030059,-0.001289,23635.929220,123566.95
954,DEU,2001.0,99801.0,3219.0,3204.0,15.0,0.032254,0.032104,0.000150,23607.882855,123566.95
955,DEU,2002.0,103074.0,3374.0,3552.0,-178.0,0.032734,0.034461,-0.001727,25077.729076,123566.95
956,DEU,2003.0,108460.0,3285.0,3446.0,-161.0,0.030288,0.031772,-0.001484,30243.576530,123566.95
957,DEU,2004.0,114841.0,2828.0,3102.0,-274.0,0.024625,0.027011,-0.002386,34044.053634,123566.95
958,DEU,2005.0,120197.0,2935.0,3338.0,-403.0,0.024418,0.027771,-0.003353,34507.368814,123566.95
959,DEU,2006.0,123553.0,2916.0,3692.0,-776.0,0.023601,0.029882,-0.006281,36323.447742,123566.95
960,DEU,2007.0,126329.0,3078.0,3766.0,-688.0,0.024365,0.029811,-0.005446,41587.212898,123566.95
961,DEU,2008.0,129338.0,3263.0,4098.0,-835.0,0.025228,0.031684,-0.006456,45427.151677,123566.95


#### Importing the country-to-country flows

In [3]:
fn_ccy = "https://tomthe.github.io/jupyterlite_migration/files/data/countrycountryyear.csv"
res = await fetch(fn_ccy)
text = await res.text()
filename = 'dfmipop.csv'
with open(filename, 'w') as f:
    f.write(text)

dfccy = pd.read_csv(filename)
del dfccy["Unnamed: 0"]
dfccy = dfccy[dfccy["n_migrations"] > 3]
dfccy["migrationfrom"] = dfccy["migrationfrom"].str.upper()
dfccy["migrationto"] = dfccy["migrationto"].str.upper()
print(dfccy.shape)
dfccy.head(10) #[df_ccy["migrationfrom"]=="deu"]

(34860, 4)


,migrationfrom,migrationto,migrationyearpadding,n_migrations
52,AFG,GBR,2016.0,4
96,AFG,USA,2010.0,4
97,AFG,USA,2011.0,5
98,AFG,USA,2012.0,6
99,AFG,USA,2013.0,5
103,AFG,USA,2017.0,8
105,AFG,USA,2019.0,5
106,AFG,USA,2020.0,14
177,ALB,ARG,2019.0,6
226,ALB,SRB,2011.0,4


In [4]:
dfplot = dfccy.groupby(["migrationfrom","migrationto"]).mean().reset_index()

## Plotting

In [20]:
# first we filter out a subset of countries with the most researchers (more than 20,000 researchers)
dfmimost = dfc.groupby("countrycode").mean().reset_index()
dfmimost = dfmimost[dfmimost["padded_population_of_researchers"]>20000]
mostcountries = dfmimost.sort_values("padded_population_of_researchers",ascending=False)["countrycode"].tolist()
dfmigdpsomeimportant = dfc[dfc["countrycode"].isin(mostcountries)]
print(len(mostcountries), mostcountries)


22 ['USA', 'CHN', 'JPN', 'GBR', 'DEU', 'ITA', 'IND', 'FRA', 'ESP', 'KOR', 'BRA', 'CAN', 'AUS', 'RUS', 'NLD', 'TUR', 'POL', 'IRN', 'SWE', 'CHE', 'BEL', 'MEX']


In [21]:
import matplotlib

In [22]:
import micropip
await micropip.install("plotly")
import plotly
import plotly.express as px

In [38]:
dfplot = dfplot[(dfplot["migrationfrom"].isin(mostcountries)) & (dfplot["migrationto"].isin(mostcountries))]
dfplot.head(5)

,migrationfrom,migrationto,migrationyearpadding,n_migrations
64,AUS,BEL,2010.090909,11.136364
67,AUS,BRA,2009.041667,10.583333
69,AUS,CAN,2009.000000,89.680000
70,AUS,CHE,2009.000000,29.000000
72,AUS,CHN,2009.000000,109.320000


In [39]:
# install and load altair
await micropip.install("altair")
import altair as alt

In [40]:
alt.Chart(dfplot).mark_rect().encode(
    x="migrationfrom:O",
    y="migrationto:O",
    color="n_migrations:Q",
    tooltip=["migrationfrom","migrationto","n_migrations"])

alt.Chart(...)

In [43]:
alt.Chart(dfc[dfc["countrycode"].isin(mostcountries)]).mark_line().encode(
    x="year:Q",
    y="padded_population_of_researchers:Q",
    color="countrycode:N",
    tooltip=["padded_population_of_researchers","countrycode","year"])

alt.Chart(...)